In [3]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 39.9 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845515 sha256=6fc5aaddb091ceecd84f5c798d3f1bf06e2e1ce75d9ae8a1d9f9a44bb764a2d0
  Stored in directory: /Users/itong1900/Library/Caches/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import ArrayType, FloatType

In [2]:
%%time
spark = SparkSession.builder.appName("ottoDB").getOrCreate()

CPU times: user 13.9 ms, sys: 16.1 ms, total: 30 ms
Wall time: 4.91 s


In [8]:
# import os
# path = "~/otto_kaggle/otto_obj_recommendation_sys/data_engineering/"
# os.getcwd()
# #os.chdir(path)

'/Users/itong1900/otto_kaggle/otto_obj_recommendation_sys/data_engineering'

In [3]:
%%time
trainDf = spark.read.json("../../train.jsonl",lineSep='\n')
testDf = spark.read.json("../../test.jsonl",lineSep='\n')

CPU times: user 6.27 ms, sys: 4.35 ms, total: 10.6 ms
Wall time: 29.2 s


In [54]:
trainDf.printSchema()

root
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: long (nullable = true)
 |    |    |-- ts: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- session: long (nullable = true)



In [4]:
## some utils
def convert_type_num(arr):
    return [0 if i == "clicks" else (1 if i == "carts" else 2) for i in arr]
convert_type_num_udf = udf(lambda row: convert_type_num(row))

def convert_ts_second_num(arr):
    return [i/1000 for i in arr]
convert_ts_seconds_udf = udf(lambda row:  convert_ts_second_num(row), ArrayType(FloatType()))

In [5]:
trainDf_transformed = trainDf.withColumn("total_action", size(col("events")))\
                             .withColumn("aids", col("events.aid"))\
                             .withColumn("ts_seconds", convert_ts_seconds_udf(col("events.ts")) )\
                             .withColumn("action_types", convert_type_num_udf(col("events.type")))\
                             .withColumn("session_start_time", col("ts_seconds").getItem(0))\
                             .withColumn("session_end_time",element_at(col('ts_seconds'), -1))\
                             .select("session", "total_action", "session_start_time", "session_end_time", "aids", "ts_seconds", "action_types")
                             

testDf_transformed = testDf.withColumn("total_action", size(col("events")))\
                          .withColumn("aids", col("events.aid"))\
                          .withColumn("ts_seconds", convert_ts_seconds_udf(col("events.ts")) )\
                          .withColumn("action_types", convert_type_num_udf(col("events.type")))\
                          .withColumn("session_start_time", col("ts_seconds").getItem(0))\
                          .withColumn("session_end_time",element_at(col('ts_seconds'), -1))\
                          .select("session", "total_action", "session_start_time", "session_end_time", "aids", "ts_seconds", "action_types")

In [54]:
import numpy as np

np.array(trainDf_transformed.select("aids").limit(20).collect())[0]

array([list([1517085, 1563459, 1309446, 16246, 1781822, 1152674, 1649869, 461689, 305831, 461689, 362233, 1649869, 1649869, 984597, 1649869, 803544, 1110941, 1190046, 1760685, 631008, 461689, 1190046, 1650637, 313546, 1650637, 979517, 351157, 1062149, 1157384, 1841388, 1469630, 305831, 1110548, 1110548, 305831, 1650114, 1604396, 1009750, 1800933, 495779, 394655, 495779, 789245, 789245, 366890, 361317, 1700164, 1755597, 789245, 784978, 1171505, 784978, 1700164, 784978, 1521766, 1725503, 528847, 1816325, 984597, 1072782, 173702, 1072782, 1407538, 1629651, 1768568, 1318324, 1840418, 1813509, 1813509, 667924, 1226444, 709550, 709417, 1225559, 1048044, 1052813, 1225559, 240346, 1582117, 1707783, 1624436, 1157411, 358305, 1202970, 832192, 1498443, 723931, 1436439, 1693461, 1206554, 1110741, 346352, 1802050, 154930, 964169, 964169, 823637, 964169, 1411683, 964169, 1167722, 964169, 1619737, 964169, 1840615, 512756, 946219, 1090479, 1164387, 1308544, 719622, 1750538, 1443747, 1750538, 337364, 1

In [60]:
%%time
aid_list = list(trainDf_transformed.limit(2).select("aids").rdd.flatMap(lambda x: x).collect())


ConnectionRefusedError: [Errno 61] Connection refused

In [49]:
[row for row in trainDf_transformed.select("aids").limit(20).collect()][0]

Row(aids=[1517085, 1563459, 1309446, 16246, 1781822, 1152674, 1649869, 461689, 305831, 461689, 362233, 1649869, 1649869, 984597, 1649869, 803544, 1110941, 1190046, 1760685, 631008, 461689, 1190046, 1650637, 313546, 1650637, 979517, 351157, 1062149, 1157384, 1841388, 1469630, 305831, 1110548, 1110548, 305831, 1650114, 1604396, 1009750, 1800933, 495779, 394655, 495779, 789245, 789245, 366890, 361317, 1700164, 1755597, 789245, 784978, 1171505, 784978, 1700164, 784978, 1521766, 1725503, 528847, 1816325, 984597, 1072782, 173702, 1072782, 1407538, 1629651, 1768568, 1318324, 1840418, 1813509, 1813509, 667924, 1226444, 709550, 709417, 1225559, 1048044, 1052813, 1225559, 240346, 1582117, 1707783, 1624436, 1157411, 358305, 1202970, 832192, 1498443, 723931, 1436439, 1693461, 1206554, 1110741, 346352, 1802050, 154930, 964169, 964169, 823637, 964169, 1411683, 964169, 1167722, 964169, 1619737, 964169, 1840615, 512756, 946219, 1090479, 1164387, 1308544, 719622, 1750538, 1443747, 1750538, 337364, 1653

In [60]:
trainDf_transformed.show(5)

+-------+------------+------------------+----------------+--------------------+--------------------+--------------------+
|session|total_action|session_start_time|session_end_time|                aids|          ts_seconds|        action_types|
+-------+------------+------------------+----------------+--------------------+--------------------+--------------------+
|      0|         276|      1.65930483E9|    1.66168499E9|[1517085, 1563459...|[1.65930483E9, 1....|[0, 0, 0, 0, 0, 0...|
|      1|          32|      1.65930483E9|    1.66171482E9|[424964, 1492293,...|[1.65930483E9, 1....|[1, 0, 1, 0, 1, 0...|
|      2|          33|      1.65930483E9|    1.66171418E9|[763743, 137492, ...|[1.65930483E9, 1....|[0, 0, 0, 0, 0, 0...|
|      3|         226|      1.65930483E9|    1.66110963E9|[1425967, 1425967...|[1.65930483E9, 1....|[1, 0, 0, 0, 1, 0...|
|      4|          19|      1.65930483E9|    1.66158669E9|[613619, 298827, ...|[1.65930483E9, 1....|[0, 0, 2, 0, 0, 0...|
+-------+------------+--

In [14]:
trainDf_transformed.show(5)

+-------+------------+--------------------+--------------------+--------------------+
|session|total_action|                aids|                  ts|                type|
+-------+------------+--------------------+--------------------+--------------------+
|      0|         276|[1517085, 1563459...|[1659304800025, 1...|[0, 0, 0, 0, 0, 0...|
|      1|          32|[424964, 1492293,...|[1659304800025, 1...|[1, 0, 1, 0, 1, 0...|
|      2|          33|[763743, 137492, ...|[1659304800038, 1...|[0, 0, 0, 0, 0, 0...|
|      3|         226|[1425967, 1425967...|[1659304800095, 1...|[1, 0, 0, 0, 1, 0...|
|      4|          19|[613619, 298827, ...|[1659304800119, 1...|[0, 0, 2, 0, 0, 0...|
+-------+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [67]:
%%time
trainDf_transformed.count()

CPU times: user 1.54 ms, sys: 1.99 ms, total: 3.53 ms
Wall time: 9.82 s


12899779

In [64]:
%%time
testDf_transformed.count()

CPU times: user 609 µs, sys: 985 µs, total: 1.59 ms
Wall time: 704 ms


1671803

In [68]:
trainDf_transformed.write.parquet("../../data/train.parquet")

In [66]:
testDf_transformed.coalesce(1).write.format("parquet").save("../../data/test.parquet")